# Prepare Proteomic Data - Intensities, REDS RBC Omics

1. Nemkov T, Stephenson D, Earley EJ, Keele GR, Hay A, Key A, Haiman ZB, Erickson C, Dzieciatkowska M, Reisz JA, Moore A, Stone M, Deng X, Kleinman S, Spitalnik SL, Hod EA, Hudson KE, Hansen KC, Palsson BO, Churchill GA, Roubinian N, Norris PJ, Busch MP, Zimring JC, Page GP, D'Alessandro A. Biological and genetic determinants of glycolysis: Phosphofructokinase isoforms boost energy status of stored red blood cells and transfusion outcomes. Cell Metab. 2024 Sep 3;36(9):1979-1997.e13. doi: 10.1016/j.cmet.2024.06.007. Epub 2024 Jul 3. PMID: 38964323; PMCID: PMC11374506.

2. D'Alessandro A, Culp-Hill R, Reisz JA, Anderson M, Fu X, Nemkov T, Gehrke S, Zheng C, Kanias T, Guo Y, Page G, Gladwin MT, Kleinman S, Lanteri M, Stone M, Busch M, Zimring JC; Recipient Epidemiology and Donor Evaluation Study-III (REDS-III). Heterogeneity of blood processing and storage additives in different centers impacts stored red blood cell metabolism as much as storage time: lessons from REDS-III-Omics. Transfusion. 2019 Jan;59(1):89-100. doi: 10.1111/trf.14979. Epub 2018 Oct 24. PMID: 30353560; PMCID: PMC6322946.

3. Josephson CD, Glynn S, Mathew S, Birch R, Bakkour S, Baumann Kreuziger L, Busch MP, Chapman K, Dinardo C, Hendrickson J, Hod EA, Kelly S, Luban N, Mast A, Norris P, Custer B, Sabino E, Sachais B, Spencer BR, Stone M, Kleinman S; National Heart, Lung, and Blood Institute (NHLBI) Recipient Epidemiology and Donor Evaluation Study-IV-Pediatric (REDS-IV-P). The Recipient Epidemiology and Donor Evaluation Study-IV-Pediatric (REDS-IV-P): A research program striving to improve blood donor safety and optimize transfusion outcomes across the lifespan. Transfusion. 2022 May;62(5):982-999. doi: 10.1111/trf.16869. Epub 2022 Apr 19. PMID: 35441384; PMCID: PMC9353062.

## Setup
### Import packages

In [1]:
import pandas as pd
from rbc_gem_utils import get_dirpath, show_versions
from rbc_gem_utils.util import AVOGADRO_NUMBER

# Show versions of notebook
show_versions()


Package Information
-------------------
rbc-gem-utils 0.0.1

Dependency Information
----------------------
beautifulsoup4                       4.12.3
bio                                   1.6.2
cobra                                0.29.0
depinfo                               2.2.0
kaleido                               0.2.1
matplotlib                            3.8.2
memote                               0.17.0
networkx                              3.2.1
notebook                              7.0.7
openpyxl                              3.1.2
pandas                                2.2.0
pre-commit                            3.6.0
pyvis                                 0.3.2
rbc-gem-utils[database,network,vis] missing
requests                             2.31.0
scipy                                1.12.0
seaborn                              0.13.2

Build Tools Information
-----------------------
pip        23.3.1
setuptools 68.2.2
wheel      0.41.2

Platform Information
-------------------

## Load RBC Proteomics
### Set organism, dataset, and paths

In [2]:
organism = "Human"
dataset_name = "REDS_RBCOmics"
proteomics_dirpath = (
    get_dirpath("proteomics", use_temp="raw") / organism / dataset_name
).resolve()
proteomics_dirpath.mkdir(exist_ok=True, parents=True)

### Load protein data

In [3]:
df_protein_data = pd.read_csv(
    proteomics_dirpath / f"{dataset_name}_ProteinData.tsv",
    sep="\t",
    index_col=None,
)
# Check to see if expected columns are included. If so, then order columns as listed.
df_protein_data = df_protein_data.loc[
    :,
    [
        "Entry",
        "Entry Name",
        "Protein",
        "Protein names",
        "Gene Names (primary)",
        "Length",
        "Mass",  # Should be in DA
    ],
]
# Sort the data via alphabetical order of protein IDs for consistency
df_protein_data = df_protein_data.sort_values(by="Entry")

df_protein_data.head()

,Entry,Entry Name,Protein,Protein names,Gene Names (primary),Length,Mass
0,A0A075B6I0,LV861_HUMAN,LV861,Immunoglobulin lambda variable 8-61,IGLV8-61,122,12814
1,A0A075B6I9,LV746_HUMAN,LV746,Immunoglobulin lambda variable 7-46,IGLV7-46,117,12468
2,A0A075B6J9,LV218_HUMAN,LV218,Immunoglobulin lambda variable 2-18,IGLV2-18,118,12412
3,A0A075B6K4,LV310_HUMAN,LV310,Immunoglobulin lambda variable 3-10,IGLV3-10,115,12441
4,A0A075B6K5,LV39_HUMAN,LV39,Immunoglobulin lambda variable 3-9,IGLV3-9,115,12332


### Set data value type and variables for columns keys 

In [4]:
protein_values_dtype = "Intensity"
sample_key = "SAMPLE ID"
donor_key = "PUBLIC DONOR ID"
time_key = "DAY"

#### Load data and map to UniProt if necessary

In [5]:
df_proteomics = pd.read_csv(
    proteomics_dirpath / f"{dataset_name}_{protein_values_dtype}Data.tsv",
    sep="\t",
    index_col=None,
)
# Transform Protein IDs to UniProt IDs
if any(df_proteomics.columns.isin(df_protein_data["Protein"])):
    df_proteomics = df_proteomics.rename(
        df_protein_data.set_index("Protein")["Entry"].to_dict(), axis=1
    )
df_proteomics

,PUBLIC DONOR ID,DAY,P02042,P69905,P68871,P02008,P02730,P30043,P32119,P02768,...,Q6E0U4,A0A075B6K4,Q02978,Q5D862,P62899,P05164,P10412,P61247,P98179,Q9UBW5
0,S001,10,221162.4375,173843.4063,88746.09375,52135.68359,49499.97656,51533.80078,28267.07422,21654.30273,...,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,S001,23,254925.4844,185570.4688,106520.92970,64307.73828,55904.87979,48715.68750,32901.54688,21783.04492,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,S001,42,132038.1250,191274.7500,103169.12500,69139.24219,52999.45138,51651.91797,30120.47461,23874.58008,...,0.0,0.0,0.000000,1.036171,0.0,0.0,0.0,0.0,0.0,0.0
3,S002,10,373217.9063,212168.9688,73410.40625,131989.10940,50343.10748,44392.84375,28602.58789,18770.19531,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,S002,23,414226.8750,222034.9844,75257.21094,133859.17190,76791.03692,39479.26953,34531.36328,24633.55078,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,S609,23,287420.7813,185876.3750,121579.28130,93143.14063,67915.18661,60063.53516,32395.58203,20457.48047,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1826,S609,42,148955.4063,190750.8438,121603.67190,88972.14063,66747.51563,60355.31250,34075.37891,21573.66602,...,0.0,0.0,1.914996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1827,S610,10,239877.7188,213266.8906,157940.76560,19776.67188,78245.21875,61375.91797,34194.27344,21734.10938,...,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1828,S610,23,166735.7344,174317.3281,77787.25000,61099.51172,51934.31250,56079.29688,30732.66992,18853.16602,...,0.0,0.0,3.967376,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


#### Sort and format data

In [6]:
index_keys = []
for label, key in zip(["donor", "timepoint"],[donor_key, time_key]):
    if key:
        index_keys += [key]
        print(f"Number of {label}s: {df_proteomics[key].nunique()}")

df = df_proteomics.set_index(index_keys)
# Sort the column data via alphabetical order of protein IDs for consistency
df = df.sort_index(axis=1).reset_index(drop=False)
# Sort the rows by Sample IDs and time points
df = df.sort_values(by=index_keys, axis=0)

df.head()

Number of donors: 610
Number of timepoints: 3


,PUBLIC DONOR ID,DAY,A0A075B6I0,A0A075B6I9,A0A075B6J9,A0A075B6K4,A0A075B6K5,A0A075B6R2,A0A075B6S5,A0A075B6S9,...,Q9Y639,Q9Y666,Q9Y696,Q9Y6B6,Q9Y6B7,Q9Y6E0,Q9Y6I3,Q9Y6M4,Q9Y6M5,Q9Y6R7
0,S001,10,114.106690,61.188179,16.998806,1.0,240.439072,0.000000,8.726534,0.000000,...,4.806035,0.000000,44.643505,130.273163,0.000000,30.318407,22.992273,0.000000,38.119408,202.467422
1,S001,23,90.694664,18.855646,0.000000,0.0,234.999283,0.000000,30.381741,148.992706,...,3.503707,0.000000,28.674469,0.000000,0.000000,33.782383,159.744751,21.729137,38.118774,210.148560
2,S001,42,123.064743,49.431454,61.730526,0.0,221.155167,0.000000,25.704966,0.000000,...,7.449622,1.478644,26.478333,139.307129,7.024934,45.400738,19.826700,22.173162,28.154861,191.642059
3,S002,10,0.000000,0.000000,0.000000,0.0,267.438843,12.034687,24.897249,0.000000,...,12.975469,0.000000,53.475803,91.111374,0.000000,50.815300,0.000000,0.000000,113.735794,264.970459
4,S002,23,64.492546,0.000000,490.452606,0.0,518.987244,0.000000,0.000000,0.000000,...,10.578275,48.285789,48.756619,13.925104,0.000000,77.841980,0.000000,0.000000,129.577789,93.992775


## Transform intensities to copy numbers
If copy numbers are provided, skip this section.

In [7]:
# Provide in picograms. Set as None to use metadata if provided
mch_sample_value = None

### Get MCH per sample
#### Load metadata corresponding to samples (optional)

In [8]:
try:
    df_metadata = pd.read_csv(
        proteomics_dirpath / f"{dataset_name}_MetaData.tsv",
        sep="\t",
        index_col=None,
    )
except FileNotFoundError:
    df_metadata = pd.DataFrame()
else:
    # Ensure only metadata corresponds to the available omics data
    if set(df_proteomics[donor_key].unique()) != set(df_metadata[donor_key].unique()):
        df_metadata = df_metadata[
            df_metadata[donor_key].isin(df_proteomics[donor_key])
        ]

df_metadata.head()

,PUBLIC DONOR ID,AS,Gender,DONDB.ABO_RH,RBCOmics.Race.Ethnicity.Group,Age,BMI,Weight,Height,Hemolysis.volume,...,Adjusted.Osmotic.Hemolysis,Adjusted.Oxidative.Hemolysis,CBC.WBC,CBC.RBC,CBC.HGB,CBC.HCT,CBC.MCV,CBC.RDW,CBC.PLT,Ferritin
0,S001,AS1,M,O+,CAUCASIAN,33,25.724339,195,73,14.0,...,51.412936,NaN,5.56,5.33,15.5,46.6,87.4,12.7,274.0,19.0
1,S002,AS3,M,A+,HIGH,71,27.891291,200,71,14.0,...,72.000942,NaN,7.32,5.38,16.5,47.9,89.0,12.7,222.0,23.0
2,S003,AS1,F,O+,HIGH,51,23.128284,139,65,11.5,...,25.647109,NaN,5.67,4.38,14.2,41.7,95.2,12.1,293.0,17.0
3,S004,AS1,F,A+,CAUCASIAN,48,23.294675,140,65,12.5,...,12.702554,NaN,6.10,4.32,13.1,40.9,94.7,12.3,376.0,21.0
4,S005,AS1,M,A+,OTHER,52,36.042480,210,64,11.0,...,55.111380,NaN,7.52,5.81,15.9,47.8,82.3,14.4,414.0,93.0


#### Get approximate dry weight for each donor

In [9]:
# Provide in picograms. Set as None to use metadata if provided
if not df_metadata.empty and mch_sample_value is None:
    try:
        MCH_per_sample = df_metadata.set_index(index_keys)
    except KeyError:
        MCH_per_sample = df_metadata.set_index(donor_key)

    MCH_per_sample = MCH_per_sample[["CBC.HGB", "CBC.RBC"]]
    MCH_per_sample = pd.Series(
        # Calculate MCH in pg using CBC.HGB and CBC.RBC measurements
        MCH_per_sample["CBC.HGB"] / MCH_per_sample["CBC.RBC"] * 10,
        name="MCH",
    )
    n_missing = len(MCH_per_sample[MCH_per_sample.isna()])
    print(
        f"Mean MCH in pg (n={len(MCH_per_sample) - n_missing}):\t{MCH_per_sample.mean():.2f}"
    )
    print(f"Missing values:\t\t{n_missing}")
    MCH_per_sample = MCH_per_sample.fillna(MCH_per_sample.mean())

elif mch_sample_value is not None:
    MCH_per_sample = pd.Series(
        [mch_sample_value] * len(df_proteomics[donor_key].unique()),
        index=df_proteomics[donor_key].unique(),
        name="MCH",
    )
    MCH_per_sample.index.name = donor_key
    print(f"Mean MCH in pg:\t{mch_sample_value:.2f}")
else:
    raise ValueError(
        "Must provide metadata containing the "
        "Mean Corpuscular Hemoglobin (MCH), or provide the value directly in picograms."
    )


# If time was not included in metadata, add as a part of index
if time_key and time_key not in MCH_per_sample.index.names:
    MCH_per_sample = (
        pd.concat(
            (
                MCH_per_sample,
                pd.Series(
                    [list(df[time_key].unique())] * len(MCH_per_sample.index),
                    index=MCH_per_sample.index,
                    name=time_key,
                ),
            ),
            axis=1,
        )
        .explode(time_key)
        .reset_index(drop=False)
    )


MCH_per_sample.to_csv(
    proteomics_dirpath / f"{dataset_name}_MCH.tsv",
    sep="\t",
    index=False
)
MCH_per_sample = MCH_per_sample.set_index(index_keys).squeeze()
MCH_per_sample.head()

Mean MCH in pg (n=598):	29.51
Missing values:		12


PUBLIC DONOR ID  DAY
S001             10     29.080675
                 23     29.080675
                 42     29.080675
S002             10     30.669145
                 23     30.669145
Name: MCH, dtype: float64

#### Transform intensities to copy numbers and expected format

In [10]:
# Convert Da to kDa
df_uniprot_to_mw = df_protein_data.set_index("Entry")["Mass"] / 1000

df_concentrations = df_proteomics.set_index(index_keys)
# Sum intensities and convert to pmol / mgDW sample
df_concentrations = (
    df_concentrations.apply(lambda x: x / x.sum(), axis=1) / df_uniprot_to_mw
)
# pmol / mgDW sample --> nmol / gDW sample
df_concentrations = df_concentrations * 1e6

# Conversion to copy numbers
df_copy_numbers = (df_concentrations * 1e-9).mul(
    MCH_per_sample * 1e-12, axis=0
) * AVOGADRO_NUMBER
df_copy_numbers

A0A075B6I0    A0A075B6I9     A0A075B6J9  A0A075B6K4  \
PUBLIC DONOR ID DAY                                                           
S001            10   114518.643899  63113.250142   17612.721227  1033.70001   
                23    83733.413360  17891.485645       0.000000     0.00000   
                42   124576.860801  51427.456139   64512.914721     0.00000   
S002            10        0.000000      0.000000       0.000000     0.00000   
                23    52980.619738      0.000000  415956.112710     0.00000   
...                            ...           ...            ...         ...   
S609            23        0.000000      0.000000       0.000000     0.00000   
                42        0.000000      0.000000       0.000000     0.00000   
S610            10    52483.854665      0.000000   16017.273082     0.00000   
                23    54229.883559      0.000000       0.000000     0.00000   
                42   129088.707067      0.000000       0.000000     0.00000   

                        A0A075B6K5    A0A075B6R2    A0A075B6S5     A0A075B6S9  \
PUBLIC DONOR ID DAY                                                             
S001            10   250738.681069      0.000000   8828.312669       0.000000   
                23   225441.984974      0.000000  28274.865365  138922.828686   
                42   232622.665201      0.000000  26229.595680       0.000000   
S002            10   246593.949891  10651.008268  22270.446851       0.000000   
                23   443011.906351      0.000000      0.000000       0.000000   
...                            ...           ...           ...            ...   
S609            23   225635.007425      0.000000   9092.998956       0.000000   
                42   138221.840018      0.000000   4613.159040       0.000000   
S610            10   149738.028981  14758.092712  10439.649393   39609.443748   
                23    93055.342976      0.000000  12750.686038   86361.815849   
                42   152987.228085      0.000000      0.000000       0.000000   

                        A0A087WSY6     A0A0A0MRZ8  ...       Q9Y639  \
PUBLIC DONOR ID DAY                                ...                
S001            10   323739.751835   61753.957120  ...  1392.454161   
                23   323562.039664   63871.782876  ...   933.842764   
                42   307083.059312   85379.086435  ...  2177.042354   
S002            10        0.000000   11125.538105  ...  3323.982341   
                23        0.000000       0.000000  ...  2508.714731   
...                            ...            ...  ...          ...   
S609            23    60754.867934   40445.049243  ...  3673.791044   
                42    88833.464684   20512.080999  ...  3465.156142   
S610            10        0.000000   20907.088540  ...  3964.128687   
                23   200688.996186  134584.998264  ...  2734.603592   
                42        0.000000   49035.593765  ...  3562.857763   

                          Q9Y666        Q9Y696         Q9Y6B6       Q9Y6B7  \
PUBLIC DONOR ID DAY                                                          
S001            10      0.000000  19954.371068   74758.901488     0.000000   
                23      0.000000  11790.361238       0.000000     0.000000   
                42    161.034294  11937.380407   80634.321987  1094.445937   
S002            10      0.000000  21133.854588   46229.818435     0.000000   
                23   4267.543219  17838.394210    6541.070445     0.000000   
...                          ...           ...            ...          ...   
S609            23      0.000000   7148.104570  124188.370607     0.000000   
                42      0.000000   7107.225657   60512.211739     0.000000   
S610            10      0.000000      0.000000   22176.084986     0.000000   
                23      0.000000  17318.853805   82170.719021     0.000000   
                42      0.000000   8789.975005       0.000000     0.000000   

   

### Export absolute quantitative data per sample

In [11]:
dataframes_dict = {
    "Concentrations": df_concentrations.reset_index(drop=False),
    "CopyNumbers": df_copy_numbers.reset_index(drop=False),
}
for data_type, df in dataframes_dict.items():
    df.to_csv(
        proteomics_dirpath / f"{dataset_name}_{data_type}.tsv", sep="\t", index=False
    )
    print(f"Saved data for {data_type}")

Saved data for Concentrations
Saved data for CopyNumbers
